In [4]:
#import re

## Process Name Functions

In [1]:
## Helper functions
def get_nickanme(x):
    import re
    if "(" in x:
        nickname = re.findall('\(.*?\)',x)[0]
    elif '"' in x:
        nickname = re.findall('\".*?\"',x)[0]
    else:
        nickname = "none"
    return nickname

def get_lastname(name, first_name, title, nickname):
    last_name = name.replace(first_name,'')
    last_name = last_name.replace(title,'')
    last_name = last_name.replace(nickname,'')
    last_name = last_name.replace(',','')
    return last_name

def replace_last(source_string, replace_what):
    head, _sep, tail = source_string.rpartition(replace_what)
    return head + "" + tail

def nlp_transforms(df):
    df.Name = df.Name.str.lower()

    ## first name, title, nickname
    df['First Name'] = df['Name'].apply(lambda x: x.split(",")[0])
    df['Title'] = df['Name'].apply(lambda x: x.split(",")[1].split()[0])
    df['Nickname'] = df['Name'].apply(lambda x: get_nickanme(x))
    df['Last Name'] = df.apply(lambda x: get_lastname(x['Name'], x['First Name'],x['Title'],x['Nickname']), axis=1)
    df['Junior'] = df['Last Name'].apply(lambda x: x.endswith(("jr","ii")))
    df['Last Name'] = df['Last Name'].apply(lambda x: replace_last(x, "jr"))
    df['Last Name'] = df['Last Name'].apply(lambda x: replace_last(x, "ii"))
    df['Last Name'] = df['Last Name'].apply(lambda x: x.strip())
    df['Middle Names'] = df['Last Name'].apply(lambda x: x.split(' ')[1:])
    df['Middle Names'] = df['Middle Names'].apply(lambda x: ' '.join(x))
    df['Last Name'] = df['Last Name'].apply(lambda x: x.split(' ')[0])
    return df


# Proccess Tickets

In [2]:
# Proccess Tickets
def replace_number(ticket, number):
    x = ticket.replace(str(number), '')
    return x

def ticket_transforms(df):
    df.Ticket = df.Ticket.str.lower()

    df['temp'] = df.Ticket.str.split()
    df['Ticket Number'] = df.temp.apply(lambda x: x[-1])
    df["Ticket Number"] = df["Ticket Number"].str.replace('line','0')
    df["Ticket Number"] = df["Ticket Number"].astype(int)
    df['Ticket Type'] = df.apply(lambda x: replace_number(x['Ticket'], x['Ticket Number']), axis=1)
    df = df.drop(['temp'],axis=1)
    return df

# Feature Selection

In [6]:
def feature_selection(drop_cols, categorical_cols, ordinal_cols):
    for column in drop_cols:
        if column in categorical_cols: categorical_cols.remove(column)
        if column in ordinal_cols: ordinal_cols.remove(column)
    features = categorical_cols + ordinal_cols
    return features,categorical_cols,ordinal_cols